In [5]:
!pip install river
%pip install sdv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 93.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.9/157.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [54]:


from river.datasets import Phishing
import pandas as pd
import numpy as np
from utils import plot_drift, generate_synthetic_from_df
from novelty import ephemeral, ephemeral_alt, persistant, periodic, degenerate

from river import datasets
from river import ensemble
from river import evaluate
from river import linear_model
from river import metrics
from river import optim
from river import preprocessing


from drift import detect_drift

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
phishing = Phishing()
list(phishing.take(2))

[({'empty_server_form_handler': 0.0,
   'popup_window': 0.0,
   'https': 0.0,
   'request_from_other_domain': 0.0,
   'anchor_from_other_domain': 0.0,
   'is_popular': 0.5,
   'long_url': 1.0,
   'age_of_domain': 1,
   'ip_in_url': 1},
  True),
 ({'empty_server_form_handler': 1.0,
   'popup_window': 0.0,
   'https': 0.5,
   'request_from_other_domain': 0.5,
   'anchor_from_other_domain': 0.0,
   'is_popular': 0.5,
   'long_url': 0.0,
   'age_of_domain': 1,
   'ip_in_url': 0},
  True)]

In [42]:
def dataset_to_df(dataset, count=None):
    if not count:
        count = len(list(dataset))
    data_list = [{**x, 'value': y} for x, y in dataset.take(count)]
    return pd.DataFrame(data_list)

def df_to_dataset(df):
    df_nonvalue = df.loc[:, df.columns != 'value']
    df_value = df['value']
    return list(zip(df_nonvalue.to_dict(orient='records'), df_value.to_list()))

In [43]:
df = dataset_to_df(phishing)
df

,empty_server_form_handler,popup_window,https,request_from_other_domain,anchor_from_other_domain,is_popular,long_url,age_of_domain,ip_in_url,value
0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,1,1,True
1,1.0,0.0,0.5,0.5,0.0,0.5,0.0,1,0,True
2,0.0,0.0,1.0,0.0,0.5,0.5,0.0,1,0,True
3,0.0,0.0,1.0,0.0,0.0,1.0,0.5,0,0,True
4,1.0,0.0,0.5,1.0,0.0,0.5,0.5,1,0,False
...,...,...,...,...,...,...,...,...,...,...
1245,0.0,0.0,0.0,0.0,0.0,0.0,0.5,1,0,True
1246,0.0,0.5,1.0,0.5,0.0,0.5,0.5,1,0,False
1247,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0,0,True
1248,0.5,0.5,1.0,0.5,0.5,0.5,0.0,1,0,True


Insert Synthetic Data to create mixed dataset

In [58]:
df_real = dataset_to_df(phishing, count=1000)
df_synth = generate_synthetic_from_df(df_real, n_samples=500)
df_mixed = pd.concat([df_real, df_synth])
df_mixed

/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:126: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:105: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


,empty_server_form_handler,popup_window,https,request_from_other_domain,anchor_from_other_domain,is_popular,long_url,age_of_domain,ip_in_url,value
0,0.0,0.0,0.0,0.0,0.0,0.5,1.0,1,1,True
1,1.0,0.0,0.5,0.5,0.0,0.5,0.0,1,0,True
2,0.0,0.0,1.0,0.0,0.5,0.5,0.0,1,0,True
3,0.0,0.0,1.0,0.0,0.0,1.0,0.5,0,0,True
4,1.0,0.0,0.5,1.0,0.0,0.5,0.5,1,0,False
...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1,0,True
996,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0,0,True
997,0.0,0.5,1.0,0.5,0.5,0.0,0.5,1,0,False
998,1.0,0.0,0.5,0.0,1.0,0.0,1.0,1,0,True


In [45]:
def eval(dataset):
    metric = metrics.F1()

    model_ADWIN = ensemble.ADWINBaggingClassifier(
        model=(
            preprocessing.StandardScaler() |
            linear_model.LogisticRegression()
        ),
        n_models=10,
        seed=42
    )

    model_plain = (
        preprocessing.StandardScaler() |
        linear_model.LogisticRegression()
    )

    ADWIN_result = evaluate.progressive_val_score(
        dataset,
        model_ADWIN,
        metric,)

    print(f"ADWIN: {ADWIN_result}")

    plain_result = evaluate.progressive_val_score(
        dataset,
        model_plain,
        metric,)

    print(f"plain: {plain_result}")

Ephemeral Novelty

In [55]:
print("Pre-Novelty Insertion")
eval(df_to_dataset(df_mixed))

print()
print("Post-Novelty Insertion")
df_ephemeral = ephemeral_alt(df_mixed, 'value', 200, 300, lambda x: not x)
eval(df_to_dataset(df_ephemeral))

Pre-Novelty Insertion
ADWIN: F1: 79.55%
plain: F1: 79.72%

Post-Novelty Insertion
ADWIN: F1: 71.40%
plain: F1: 71.58%


In [56]:
def eval_Ada(dataset):
    model_Ada = ensemble.AdaBoostClassifier(
        model=(
            preprocessing.StandardScaler() |
            linear_model.LogisticRegression()
        ),
        n_models=5,

    )

    model_plain = (
        preprocessing.StandardScaler() |
        linear_model.LogisticRegression()
    )

    metric = metrics.F1()

    Ada_result = evaluate.progressive_val_score(
        dataset,
        model_Ada,
        metric,)

    print(f"Ada: {Ada_result}")


    plain_result = evaluate.progressive_val_score(
        dataset,
        model_plain,
        metric,)

    print(f"plain: {plain_result}")

In [57]:
print("Pre-Novelty")
eval_Ada(df_to_dataset(df_mixed))

print()
print("Post-Novelty Insertion")
eval_Ada(df_to_dataset(df_mixed))

Pre-Novelty
Ada: F1: 77.72%
plain: F1: 74.64%

Post-Novelty Insertion
Ada: F1: 79.02%
plain: F1: 75.34%


Insert Synthetic Data to create purely synthetic dataset

In [26]:
df_real = dataset_to_df(phishing, count=1000)
df_synth = generate_synthetic_from_df(df_real, n_samples=500)
df_synth

/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:126: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:105: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


,empty_server_form_handler,popup_window,https,request_from_other_domain,anchor_from_other_domain,is_popular,long_url,age_of_domain,ip_in_url,value
0,0.4,0.5,0.8,0.8,1.0,0.5,0.1,1,0,False
1,0.8,0.1,0.6,1.0,1.0,0.9,0.0,0,0,False
2,0.1,0.5,0.7,0.0,0.0,0.8,1.0,0,0,True
3,0.1,0.1,0.0,0.0,0.0,0.4,0.0,0,0,True
4,0.0,0.1,0.8,0.4,0.0,0.1,0.8,1,1,False
...,...,...,...,...,...,...,...,...,...,...
495,0.0,0.0,0.1,0.3,0.4,0.4,0.0,1,0,True
496,0.8,0.8,1.0,1.0,0.0,0.0,0.8,1,0,False
497,0.9,0.7,0.4,1.0,0.9,0.0,0.1,1,0,False
498,0.0,0.1,0.0,0.3,0.1,0.9,0.4,0,0,True


In [27]:
def eval(dataset):
    metric = metrics.F1()

    model_ADWIN = ensemble.ADWINBaggingClassifier(
        model=(
            preprocessing.StandardScaler() |
            linear_model.LogisticRegression()
        ),
        n_models=10,
        seed=42
    )

    model_plain = (
        preprocessing.StandardScaler() |
        linear_model.LogisticRegression()
    )

    ADWIN_result = evaluate.progressive_val_score(
        dataset,
        model_ADWIN,
        metric,)

    print(f"ADWIN: {ADWIN_result}")

    plain_result = evaluate.progressive_val_score(
        dataset,
        model_plain,
        metric,)

    print(f"plain: {plain_result}")

In [39]:
print("Pre-Novelty Insertion")
eval(df_to_dataset(df_synth))

print()
print("Post-Novelty Insertion")
df_ephemeral = ephemeral(df_synth, 'value', 200, 300, lambda x: x+5)
eval(df_to_dataset(df_ephemeral))

Pre-Novelty Insertion
ADWIN: F1: 63.11%
plain: F1: 63.40%

Post-Novelty Insertion


/content/novelty.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[6 5 5 6 6 5 5 5 5 5 6 5 5 5 6 5 5 5 6 5 5 5 5 5 6 6 5 5 6 5 5 5 5 5 5 5 6
 6 6 6 5 5 5 5 6 5 6 6 6 6 6 5 5 5 6 6 5 5 5 6 6 5 5 6 6 5 5 5 6 5 5 6 5 5
 6 5 5 5 6 5 5 5 5 5 5 6 5 5 6 6 5 5 6 5 6 5 6 5 5 6 5 5 5 5 6 6 5 6 5 6 5
 6 6 5 5 6 5 6 5 5 5 5 6 5 6 6 6 5 5 6 5 5 5 5 5 5 5 5 5 5 5 5 6 5 5 6 5 6
 5 5 6 5 6 5 6 6 6 5 5 6 5 5 5 5 6 6 6 6 6 5 6 6 5 6 6 5 5 5 6 5 6 5 6 5 6
 6 5 6 5 5 5 5 5 6 5 5 5 6 5 5 6 6 5 5 5 5 5 6 6 6 5 5 5 5 6 6 5 5 5 6 5 5
 6 6 6 5 5 5 5 5 6 5 5 6 5 5 6 5 5 6 5 5 5 5 5 5 5 6 6 5 5 5 5 5 6 5 5 5 6
 5 6 6 5 5 5 5 6 5 5 5 5 6 5 5 6 5 6 5 6 6 6 6 5 5 6 5 5 5 5 5 5 6 5 6 5 6
 5 5 6 5]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  data.loc[start : end, feature_name] = data.loc[start : end, feature_name].apply(func)


ADWIN: F1: 26.12%
plain: F1: 26.56%


In [40]:
def eval_Ada(dataset):
    model_Ada = ensemble.AdaBoostClassifier(
        model=(
            preprocessing.StandardScaler() |
            linear_model.LogisticRegression()
        ),
        n_models=5,

    )

    model_plain = (
        preprocessing.StandardScaler() |
        linear_model.LogisticRegression()
    )

    metric = metrics.F1()

    Ada_result = evaluate.progressive_val_score(
        dataset,
        model_Ada,
        metric,)

    print(f"Ada: {Ada_result}")


    plain_result = evaluate.progressive_val_score(
        dataset,
        model_plain,
        metric,)

    print(f"plain: {plain_result}")

In [41]:
print("Pre-Novelty")
eval(df_to_dataset(df_synth))

print()
print("Post-Novelty Insertion")
eval_Ada(df_to_dataset(df_ephemeral))

Pre-Novelty
ADWIN: F1: 63.11%
plain: F1: 63.40%

Post-Novelty Insertion
Ada: F1: 25.81%
plain: F1: 26.40%
